In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data

In [3]:
#Load train data
X_train = pd.read_csv('data/EstrogenReceptorStatus_Train.csv',index_col=0)
print("Training shape: {}".format(X_train.shape))

y_train = pd.read_csv('data/EstrogenReceptorStatus_Train_labels.txt',header=None)
print("Training labels shape: {}".format(y_train.shape))
print("Examples per label")
print(y_train.groupby([0])[0].count())

print("Training data head")
print(X_train.head(3))
# X_train = X_train.to_numpy
X_train = np.asarray(X_train)

print("Training labels head")
print(y_train.head(3))
# y_train = y_train.to_numpy
y_train = np.asarray(y_train)


Training shape: (216, 162)
Training labels shape: (216, 1)
Examples per label
0
0     55
1    161
Name: 0, dtype: int64
Training data head
         1-hexadecanol  1-monostearin  1,2,4-benzenetriol  \
case_95       9.729131       9.372530           12.635429   
case_88      10.961073       9.353313           12.692373   
case_24      10.995057       9.384110           12.954097   

         1,5-anhydroglucitol  2-aminoadipic,acid  2-hydroxybutanoic,acid  \
case_95            14.711658           11.162607               14.145654   
case_88            14.371407           11.128662               14.228230   
case_24            13.323175            8.162313               14.392132   

         2-hydroxyglutaric,acid  2-hydroxyvaleric,acid  2-ketoadipic,acid  \
case_95               12.334462              14.358346          12.103635   
case_88               12.177452              14.923601          12.378531   
case_24               17.532062              16.000497          12.558596   

  

In [4]:
#Load test data
X_test = pd.read_csv('data/EstrogenReceptorStatus_Test.csv',index_col=0)
print("Test shape: {}".format(X_test.shape))
# X_test = X_test.to_numpy
X_test = np.asarray(X_test)
print("Type: {}".format(type(X_test)))

y_test = pd.read_csv('data/EstrogenReceptorStatus_Test_labels.txt',header=None)
print("Test labels shape: {}".format(y_test.shape))
print("Examples per label")
print(y_test.groupby([0])[0].count())
# y_test = y_test.to_numpy
y_test = np.asarray(y_test)


Test shape: (55, 162)
Type: <class 'numpy.ndarray'>
Test labels shape: (55, 1)
Examples per label
0
0    12
1    43
Name: 0, dtype: int64


In [5]:
# Mapeamos datos a tensores de PyTorch
X_train, y_train, X_test, y_test = map(torch.tensor, (X_train, y_train, X_test, y_test))
y_train = torch.flatten(y_train)
y_test = torch.flatten(y_test)
print("Training data shape: {}".format(X_train.shape))
print("Training label shape: {}".format(y_train.shape))
print("Test data shape: {}".format(X_test.shape))
print("Test label shape: {}".format(y_test.shape))

Training data shape: torch.Size([216, 162])
Training label shape: torch.Size([216])
Test data shape: torch.Size([55, 162])
Test label shape: torch.Size([55])


In [6]:
X_train = X_train.float()
y_train = y_train.long()
X_test = X_test.float()
y_test = y_test.long()

print("Training data shape: {}".format(X_train.shape))
print("Training label shape: {}".format(y_train.shape))
print("Test data shape: {}".format(X_test.shape))
print("Training data shape: {}".format(y_test.shape))

Training data shape: torch.Size([216, 162])
Training label shape: torch.Size([216])
Test data shape: torch.Size([55, 162])
Training data shape: torch.Size([55])


In [7]:
class FFNN(nn.Module):
  
  def __init__(self):
    super().__init__()
    torch.manual_seed(0)
    self.net = nn.Sequential(
        nn.Linear(162, 30), 
        nn.ReLU(),
        nn.Linear(30, 30),
        nn.ReLU(),
        nn.Linear(30, 2), 
        nn.Softmax()
    )

  def forward(self, X):
    return self.net(X)

  def predict(self, X):
    Y_pred = self.forward(X)
    return Y_pred


In [8]:
def fit_v2(x, y, model, opt, loss_fn, epochs = 1000):
  
  for epoch in range(epochs):
    loss = loss_fn(model(x), y)
    loss.backward()
    opt.step()
    opt.zero_grad()
    #print(loss.item())
    
  return loss.item()

In [9]:
device = torch.device("cpu")

X_train=X_train.to(device)
y_train=y_train.to(device)
X_test=X_test.to(device)
y_test=y_test.to(device)
fn = FFNN()
fn.to(device)
loss_fn = F.cross_entropy
opt = optim.SGD(fn.parameters(), lr=0.01)
# opt = optim.AdamW(fn.parameters(), lr=0.01)

print('Final loss', fit_v2(X_train, y_train, fn, opt, loss_fn))

0.7035389542579651
0.6246233582496643
0.5970936417579651
0.5865880250930786
0.5811102390289307
0.5778590440750122
0.5757544636726379
0.5742765665054321
0.5731554627418518
0.5722631812095642
0.5715357661247253
0.5709974765777588
0.5705819129943848
0.5702391862869263
0.5699617862701416
0.5697269439697266
0.5695198774337769
0.5693420767784119
0.5691863894462585
0.5690478682518005
0.5689261555671692
0.5688229203224182


0.5687313079833984
0.5686507225036621
0.568577766418457
0.5685119032859802
0.5684515833854675
0.5683973431587219
0.568348228931427
0.5683044791221619
0.5682637691497803
0.5682268738746643
0.5681917071342468
0.568159282207489
0.5681290030479431
0.5681004524230957
0.5680734515190125
0.5680480003356934
0.5680241584777832
0.5680018067359924
0.5679810643196106
0.5679613947868347
0.56794273853302
0.5679249167442322
0.567907989025116
0.5678921341896057
0.5678770542144775
0.5678627490997314
0.5678490996360779
0.5678361058235168
0.5678237080574036
0.5678118467330933
0.5678006410598755
0.5677897930145264
0.5677793025970459
0.5677690505981445
0.5677592158317566
0.5677497386932373
0.5677406191825867
0.5677317976951599
0.5677233338356018
0.5677151083946228
0.5677071809768677
0.567699670791626


c:\Users\COMPUTOCKS\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.5676922798156738
0.5676852464675903
0.5676780343055725
0.5676708817481995
0.5676636695861816
0.5676567554473877
0.5676495432853699
0.5676427483558655
0.5676358938217163
0.567629337310791
0.5676214098930359
0.5676133632659912
0.5676058530807495
0.5675987601280212
0.567592203617096
0.5675857067108154
0.5675793290138245
0.5675718188285828
0.5675636529922485
0.5675562024116516
0.5675480365753174
0.5675399303436279
0.5675321221351624
0.5675253868103027
0.5675185918807983
0.5675120949745178
0.5675055384635925
0.5674991011619568
0.5674921274185181
0.5674852132797241
0.5674787759780884
0.5674728751182556
0.5674668550491333
0.5674605965614319
0.5674545168876648
0.567448079586029
0.567441463470459
0.5674351453781128
0.5674291253089905
0.5674232840538025
0.5674178600311279
0.5674126744270325
0.5674073696136475
0.5674020648002625
0.567396879196167
0.5673916935920715
0.5673863887786865
0.5673815608024597
0.5673770904541016
0.5673725008964539
0.5673672556877136
0.5673620700836182
0.567357182502746

In [10]:
def accuracy(y_hat, y):
  pred = torch.argmax(y_hat, dim=1)
  return (pred == y).float().mean()

In [11]:
Y_pred_train = fn.predict(X_train)
#Y_pred_train = np.argmax(Y_pred_train,1)
Y_pred_test = fn.predict(X_test)
#Y_pred_val = np.argmax(Y_pred_val,1)

accuracy_train = accuracy(Y_pred_train, y_train)
accuracy_test = accuracy(Y_pred_test, y_test)

print("Training accuracy", (accuracy_train))
print("Test accuracy",(accuracy_test))

Training accuracy tensor(0.7454)
Test accuracy tensor(0.7818)


In [12]:
print(Y_pred_test)

tensor([[1.4470e-03, 9.9855e-01],
        [1.6987e-03, 9.9830e-01],
        [7.4396e-04, 9.9926e-01],
        [6.4093e-04, 9.9936e-01],
        [7.4992e-04, 9.9925e-01],
        [2.3222e-03, 9.9768e-01],
        [4.2696e-04, 9.9957e-01],
        [2.0166e-04, 9.9980e-01],
        [5.2661e-04, 9.9947e-01],
        [4.1780e-04, 9.9958e-01],
        [2.1103e-03, 9.9789e-01],
        [8.0393e-04, 9.9920e-01],
        [4.0256e-04, 9.9960e-01],
        [3.5450e-04, 9.9965e-01],
        [2.9960e-03, 9.9700e-01],
        [7.3735e-04, 9.9926e-01],
        [1.4495e-03, 9.9855e-01],
        [1.9574e-03, 9.9804e-01],
        [4.7706e-04, 9.9952e-01],
        [4.5524e-04, 9.9954e-01],
        [2.1890e-03, 9.9781e-01],
        [2.9192e-04, 9.9971e-01],
        [3.2595e-04, 9.9967e-01],
        [2.0867e-03, 9.9791e-01],
        [9.5795e-04, 9.9904e-01],
        [3.0283e-04, 9.9970e-01],
        [1.2642e-03, 9.9874e-01],
        [6.2599e-03, 9.9374e-01],
        [9.6212e-04, 9.9904e-01],
        [6.424

In [13]:
pred = torch.argmax(Y_pred_test, dim=1)
print(pred)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1])
